In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler 
#from sklearn.preprocessing import RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.model_selection import train_test_split, cross_val_score
#from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from datetime import datetime

In [2]:
df = pd.read_csv('Preprocessed_data.csv', index_col = 0)
df

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_binary
0,0,0,1,23,0,0,0,1,0,0,...,0,1,0,0,0,0,11,5,7,0
1,1,0,1,19,0,0,0,0,1,1,...,0,3,0,0,0,0,6,6,8,0
2,0,0,1,26,1,0,0,1,1,1,...,0,2,0,0,0,0,1,4,4,0
3,0,1,1,22,0,0,0,1,1,1,...,0,1,0,0,0,1,12,4,2,0
4,0,0,1,22,0,0,0,0,1,1,...,0,1,0,0,0,0,4,6,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253659,0,1,1,37,0,0,0,0,0,1,...,0,4,0,0,0,0,6,4,1,1
253668,0,1,1,29,1,0,1,0,1,1,...,0,2,0,0,1,1,10,3,6,1
253670,1,1,1,25,0,0,1,0,1,0,...,0,5,15,0,1,0,13,6,4,1
253676,1,1,1,18,0,0,0,0,0,0,...,0,4,0,0,1,0,11,2,4,1


In [3]:
X = df.iloc[:,0:-1]
y = df.loc[:, 'Diabetes_binary']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 1)

In [4]:
acc_list = []
roc_auc_ovr_List = []
f1_micro_List = []

## Trial 1

In [5]:
start = datetime.now()

In [7]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0, penalty = )

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [8]:
best_model.cv_results_

{'mean_fit_time': array([0.65914841, 0.65778089, 0.5563247 , 0.8201467 , 0.85659654,
        0.61211035, 0.84067125, 0.88029273, 0.83573036, 0.85770466,
        0.73098173, 0.89634936, 0.98322189, 0.71891797, 0.71105556,
        0.75074055, 0.86792619, 0.770857  , 0.09161048, 0.10807879,
        0.10175478, 0.10385723, 0.10838196, 0.10472271, 0.10932174,
        0.10816913, 0.10585136, 0.10269229, 0.8219702 , 0.11308763]),
 'std_fit_time': array([0.14267239, 0.14577944, 0.0861196 , 0.20688543, 0.26137992,
        0.22976913, 0.30814208, 0.26009263, 0.29695248, 0.26156124,
        0.17729057, 0.28477449, 0.28926526, 0.28346317, 0.1607101 ,
        0.07805331, 0.0922573 , 0.02717971, 0.00471796, 0.00423326,
        0.0045063 , 0.00572735, 0.00066364, 0.00278133, 0.009425  ,
        0.00071958, 0.00627034, 0.00563807, 0.04701035, 0.00360152]),
 'mean_score_time': array([0.01089888, 0.01220818, 0.00938954, 0.01338258, 0.01389461,
        0.01270204, 0.01507306, 0.01168995, 0.01257172, 0.01

In [9]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27,  3,  5,  1, 26, 23, 25,  6, 17, 14,  6,  6,  6,  6, 14,
       14, 28,  3,  2, 23, 12, 18, 18, 18, 18, 18,  6, 12])

In [10]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [11]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.600535
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.738910
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.744299
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.746648
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.746647
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.746704
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746563
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.746577
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.746577
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.746619


In [12]:
acc_list.append(results[0:1])

In [13]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [14]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols


results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.712024
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.699787
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.698351
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.695671
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.695219
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.694857
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.694780
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.694794
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.694844


In [15]:
roc_auc_ovr_List.append(results[1:2])

In [16]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.01,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [17]:
results = pd.DataFrame( best_model.cv_results_['params'] )


results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.600535
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.738910
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.744299
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.746648
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.746647
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.746704
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746563
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.746577
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.746577
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.746619


In [18]:
f1_micro_List.append(results[5:6])

## Trial 2

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 2)

In [20]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [21]:
best_model.cv_results_

{'mean_fit_time': array([0.98096454, 0.74885468, 1.06254656, 0.73179348, 0.87304895,
        0.79987073, 0.89625387, 0.68071721, 0.58893275, 0.62166436,
        0.55435581, 0.46965127, 0.76578488, 0.46524472, 0.56083415,
        0.4551805 , 0.53803239, 0.56402125, 0.08131628, 0.09206002,
        0.09261935, 0.09434807, 0.09456151, 0.08888946, 0.08935623,
        0.09464142, 0.09537334, 0.09137297, 0.46612024, 0.09281585]),
 'std_fit_time': array([0.2530577 , 0.21492931, 0.21009976, 0.26251785, 0.2686503 ,
        0.25128728, 0.22592188, 0.25689996, 0.1213885 , 0.14629934,
        0.037988  , 0.02467861, 0.16026908, 0.02075705, 0.02352041,
        0.02240403, 0.0248996 , 0.14583097, 0.00087406, 0.00404896,
        0.00937839, 0.00234774, 0.00209957, 0.0021123 , 0.0087712 ,
        0.00272986, 0.00196472, 0.00229208, 0.02875863, 0.00227822]),
 'mean_score_time': array([0.02022383, 0.01636581, 0.01615379, 0.01436884, 0.0155673 ,
        0.01577425, 0.01270113, 0.01619236, 0.00986848, 0.01

In [22]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 24, 26, 22, 21, 20, 13, 18,  2,  2,  2,  2,  2,  1,  2,
       14, 29, 24, 22, 19, 16, 15,  2,  2,  2,  2,  2, 16])

In [23]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [24]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.599120
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.739009
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.743648
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.746265
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.746096
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.746718
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746930
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.746930
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.747029
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.747001


In [25]:
acc_list.append(results[0:1])

In [26]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [27]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.712332
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.699097
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.697913
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.694974
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.695491
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.695458
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.695399
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.695463
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.695412


In [28]:
roc_auc_ovr_List.append(results[1:2])

In [29]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 1000.0,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [30]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.599120
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.739009
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.743648
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.746265
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.746096
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.746718
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746930
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.746930
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.747029
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.747001


In [31]:
f1_micro_List.append(results[15:16])

## Trial 3

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 3)

In [33]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [34]:
best_model.cv_results_

{'mean_fit_time': array([0.75290251, 0.66871369, 0.76771591, 0.65874445, 0.81238613,
        0.71601481, 0.89895306, 0.86977723, 1.02176297, 0.87500408,
        0.96130064, 0.88397105, 0.96545217, 0.90873976, 0.9930603 ,
        0.92028687, 0.86143777, 0.70369284, 0.09372852, 0.10877235,
        0.10553417, 0.11161778, 0.10928605, 0.10749404, 0.10272226,
        0.10854416, 0.10725744, 0.10935886, 0.71608827, 0.10303392]),
 'std_fit_time': array([0.1164699 , 0.03797113, 0.04754982, 0.06160515, 0.0830729 ,
        0.05015624, 0.20079105, 0.14390754, 0.21167597, 0.17702368,
        0.23219803, 0.2168387 , 0.2156959 , 0.18614958, 0.21113671,
        0.22258151, 0.07744749, 0.04635152, 0.00919033, 0.00509653,
        0.0055913 , 0.01758602, 0.00523933, 0.0053457 , 0.0047531 ,
        0.01160619, 0.0064342 , 0.00523853, 0.04525995, 0.00603959]),
 'mean_score_time': array([0.01372344, 0.01285002, 0.01133633, 0.01251369, 0.01313908,
        0.01365538, 0.01288407, 0.01837037, 0.01400797, 0.01

In [35]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 26, 24, 22,  1, 10,  2, 12, 12, 12, 12, 12, 12, 21, 12,
       12, 28, 25, 23, 10,  3,  3,  3,  3,  3,  3, 12,  3])

In [36]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [37]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.599601
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.738598
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.744144
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.745841
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.746082
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.746548
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746845
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.746732
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.746746
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.746718


In [38]:
acc_list.append(results[0:1])

In [39]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [40]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.711806
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.700039
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.697754
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.695360
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.695721
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.695813
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.695562
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.695544
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.695462


In [41]:
roc_auc_ovr_List.append(results[1:2])

In [42]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.1,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [43]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.599601
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.738598
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.744144
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.745841
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.746082
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.746548
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746845
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.746732
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.746746
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.746718


In [44]:
f1_micro_List.append(results[6:7])

## Trial 4

In [45]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 4)

In [46]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [47]:
best_model.cv_results_

{'mean_fit_time': array([0.74817042, 0.83071849, 0.92255409, 0.82142038, 0.94145184,
        0.73918786, 0.8548209 , 0.78852119, 0.93946428, 0.89510469,
        0.85916581, 0.72617605, 0.80873239, 0.7384202 , 0.85223684,
        0.7268471 , 0.83755894, 0.73307202, 0.092466  , 0.10507827,
        0.11189511, 0.10659344, 0.10310664, 0.10507123, 0.10861759,
        0.10577168, 0.10335834, 0.10896537, 0.66941097, 0.10435119]),
 'std_fit_time': array([0.09641451, 0.19576579, 0.2168164 , 0.20265125, 0.1898917 ,
        0.11603264, 0.21304173, 0.18510777, 0.21696737, 0.13338066,
        0.14125117, 0.05174776, 0.06571516, 0.04711891, 0.04958419,
        0.06313497, 0.08856912, 0.07403344, 0.00538718, 0.00497292,
        0.01822604, 0.00518751, 0.00474961, 0.00865724, 0.00812979,
        0.00742767, 0.00487715, 0.0103398 , 0.05389684, 0.00552636]),
 'mean_score_time': array([0.01286085, 0.01367836, 0.01758161, 0.01593397, 0.01870573,
        0.01962535, 0.01059575, 0.01865206, 0.01605554, 0.01

In [48]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 25, 24, 21, 23, 19,  4, 18,  5,  5,  5,  5,  5,  5,  5,
        5, 28, 25, 22, 20,  1,  5,  5,  5,  5,  5,  1,  1])

In [49]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [50]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.599332
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.739235
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.743663
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.746393
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.747001
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.747539
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.747524
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.747623
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.747652
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.747638


In [51]:
acc_list.append(results[0:1])

In [52]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [53]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.712440
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.699038
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.698542
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.696525
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.697094
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.696798
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.696894
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.696886
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.696889


In [54]:
roc_auc_ovr_List.append(results[1:2])

In [55]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 1.0,
 'classifier__penalty': 'l2',
 'classifier__solver': 'lbfgs'}

In [56]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.599332
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.739235
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.743663
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.746393
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.747001
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.747539
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.747524
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.747623
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.747652
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.747638


In [57]:
f1_micro_List.append(results[22:23])

## Trial 5

In [58]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 5)

In [59]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [60]:
best_model.cv_results_

{'mean_fit_time': array([0.78632846, 0.69170606, 0.76409299, 0.70536613, 0.80667536,
        0.67185853, 0.82659299, 0.707657  , 0.8070874 , 0.75679822,
        0.84421754, 0.76328862, 0.90275533, 0.75771015, 1.03116047,
        0.90111012, 1.03263774, 0.85767303, 0.16159616, 0.10001793,
        0.16521969, 0.11602077, 0.15887327, 0.1203758 , 0.14901106,
        0.14540467, 0.11140745, 0.18314481, 0.92291272, 0.1095053 ]),
 'std_fit_time': array([0.08007347, 0.06232404, 0.06881341, 0.0450978 , 0.06945838,
        0.05971017, 0.07673319, 0.05602309, 0.09284077, 0.04248772,
        0.05653993, 0.05518273, 0.06264076, 0.15162162, 0.2047497 ,
        0.25847903, 0.28298604, 0.20248017, 0.04727136, 0.01287666,
        0.07455344, 0.02874773, 0.07616799, 0.02222603, 0.06828156,
        0.0529832 , 0.02575629, 0.06630067, 0.23659766, 0.01350251]),
 'mean_score_time': array([0.01278284, 0.01367896, 0.01289692, 0.01204021, 0.01356025,
        0.013661  , 0.01358058, 0.0112427 , 0.0136925 , 0.01

In [61]:
best_model.cv_results_['rank_test_accuracy']

array([30, 29, 27, 25, 24,  2,  1,  5,  4,  7, 23,  7,  7,  7,  7,  7, 22,
        7, 28, 25,  2,  5,  7,  7,  7,  7,  7,  7,  7,  7])

In [62]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [63]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.600167
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.738188
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.743450
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.746266
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.746350
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.746944
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746987
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.746930
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.746944
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.746916


In [64]:
acc_list.append(results[0:1])

In [65]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'lbfgs'}

In [66]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.711224
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.698994
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.698263
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.695633
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.696157
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.695861
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.695828
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.695825
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.695773


In [67]:
roc_auc_ovr_List.append(results[18:19])

In [68]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.1,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [69]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.600167
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.738188
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.743450
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.746266
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.746350
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.746944
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746987
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.746930
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.746944
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.746916


In [70]:
f1_micro_List.append(results[6:7])

## Trial 6

In [71]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 6)

In [72]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [73]:
best_model.cv_results_

{'mean_fit_time': array([0.7238282 , 0.68301589, 0.78766301, 0.68140607, 0.93995371,
        0.8689219 , 0.94830196, 0.86871164, 0.95786221, 0.82102621,
        0.98706701, 0.94717734, 1.00512938, 0.86453068, 0.84875662,
        0.74552827, 0.84238129, 0.73053272, 0.0881928 , 0.10968409,
        0.10658653, 0.1032645 , 0.11247077, 0.11144149, 0.10793869,
        0.11058743, 0.11037636, 0.10850587, 0.70750315, 0.10802708]),
 'std_fit_time': array([0.05273963, 0.0525078 , 0.05275092, 0.0526618 , 0.19288239,
        0.1593121 , 0.23436877, 0.21568159, 0.23249995, 0.21485933,
        0.15452449, 0.23078503, 0.18278039, 0.19824046, 0.04588361,
        0.0550034 , 0.04550636, 0.02985085, 0.00176807, 0.01031615,
        0.0030262 , 0.00489431, 0.0152273 , 0.00840067, 0.0069901 ,
        0.00601162, 0.01008524, 0.00521619, 0.06967234, 0.00573974]),
 'mean_score_time': array([0.01214473, 0.01318908, 0.01283658, 0.01276052, 0.01828327,
        0.01596189, 0.01122289, 0.01884975, 0.01533678, 0.01

In [74]:
best_model.cv_results_['rank_test_accuracy']

array([30, 29, 27, 26, 22, 23, 21, 10, 17,  2,  2, 11,  2,  2,  2,  2,  1,
        2, 28, 25, 24, 17, 19, 11, 11, 11, 11, 11,  2, 19])

In [75]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [76]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.599389
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.738047
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.743337
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.745587
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.746209
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.746152
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746351
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.746379
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.746365
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.746393


In [77]:
acc_list.append(results[0:1])

In [78]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'lbfgs'}

In [79]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.711124
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.699183
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.697609
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.695848
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.695464
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.695405
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.695462
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.695377
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.695416


In [80]:
roc_auc_ovr_List.append(results[18:19])

In [81]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 10000.0,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [82]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.599389
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.738047
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.743337
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.745587
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.746209
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.746152
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746351
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.746379
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.746365
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.746393


In [83]:
f1_micro_List.append(results[16:17])

## Trial 7

In [84]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 7)

In [85]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [86]:
best_model.cv_results_

{'mean_fit_time': array([0.79539828, 0.70283976, 0.75831919, 0.71503546, 0.81088836,
        0.66977479, 0.79641614, 0.70065887, 0.84632592, 0.72151012,
        0.81058123, 0.71849756, 0.86259899, 0.81215818, 1.02216477,
        0.88454225, 1.08990109, 0.82144222, 0.13285568, 0.10635614,
        0.19018342, 0.10386753, 0.1802973 , 0.11456366, 0.16782601,
        0.12415531, 0.14035251, 0.15104771, 0.88259692, 0.12138464]),
 'std_fit_time': array([0.06473928, 0.05706672, 0.06677237, 0.04451318, 0.04502492,
        0.07361829, 0.06383502, 0.06927684, 0.07096526, 0.05831116,
        0.06705389, 0.03117345, 0.07805396, 0.15051027, 0.20421449,
        0.23976828, 0.18383775, 0.15933493, 0.05272069, 0.00847073,
        0.04398974, 0.00406263, 0.06897117, 0.02799582, 0.07164384,
        0.02543507, 0.06170585, 0.05423636, 0.19641659, 0.03158882]),
 'mean_score_time': array([0.01314368, 0.01248751, 0.01442001, 0.01354373, 0.01323316,
        0.01533308, 0.01185451, 0.01202853, 0.01283832, 0.01

In [87]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 25, 24, 22,  1,  2,  5, 10, 16, 10, 10,  6,  6,  4, 10,
       10, 28, 25, 22,  2,  8, 17, 17, 17, 17, 17, 10,  8])

In [88]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [89]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.599785
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.737806
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.742050
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.745544
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.745742
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.746251
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746464
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.746449
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.746393
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.746365


In [90]:
acc_list.append(results[0:1])

In [91]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [92]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.711292
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.697908
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.697841
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.695643
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.695706
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.695531
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.695583
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.695415
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.695377


In [93]:
roc_auc_ovr_List.append(results[1:2])

In [94]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.1,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [95]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.599785
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.737806
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.742050
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.745544
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.745742
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.746251
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746464
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.746449
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.746393
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.746365


In [96]:
f1_micro_List.append(results[6:7])

## Trial 8

In [97]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 8)

In [98]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [99]:
best_model.cv_results_

{'mean_fit_time': array([0.75963612, 0.70419698, 0.73650377, 0.6975759 , 0.78547673,
        0.72312765, 1.01477489, 0.86022205, 1.10970495, 0.85233052,
        0.99277034, 0.83934462, 1.03166394, 0.85490468, 1.04923179,
        0.75445869, 0.78232036, 0.72454593, 0.0917403 , 0.10399435,
        0.10750997, 0.1098058 , 0.10713108, 0.10839021, 0.10747228,
        0.10636795, 0.10876765, 0.10564013, 0.72149885, 0.10869601]),
 'std_fit_time': array([0.07955152, 0.07005131, 0.05341697, 0.05611961, 0.07649789,
        0.16172454, 0.28443228, 0.20647911, 0.24805301, 0.20393432,
        0.18884795, 0.19014833, 0.25463941, 0.21010194, 0.22602989,
        0.15440629, 0.06949143, 0.0451283 , 0.00391602, 0.00558546,
        0.01178968, 0.00832468, 0.00559549, 0.00691205, 0.01296543,
        0.00776117, 0.00541714, 0.00554944, 0.04602745, 0.00543743]),
 'mean_score_time': array([0.01193368, 0.01313634, 0.01193237, 0.0121654 , 0.0132374 ,
        0.01607313, 0.01489971, 0.01282377, 0.01726813, 0.02

In [100]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 25, 24, 21, 21, 19,  1,  2,  5,  5,  5,  5,  5,  5,  5,
        5, 28, 25, 21, 20,  2,  5,  5,  5,  5,  5, 18,  2])

In [101]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [102]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.599969
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.737693
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.742828
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.745304
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.745403
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.745912
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.745912
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.746082
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.746138
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.746138


In [103]:
acc_list.append(results[0:1])

In [104]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [105]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.710429
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.698157
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.696882
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.694410
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.694563
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.694289
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.694514
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.694545
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.694545


In [106]:
roc_auc_ovr_List.append(results[1:2])

In [107]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 1.0,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [108]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.599969
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.737693
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.742828
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.745304
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.745403
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.745912
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.745912
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.746082
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.746138
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.746138


In [109]:
f1_micro_List.append(results[8:9])

## Trial 9

In [110]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 9)

In [111]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [112]:
best_model.cv_results_

{'mean_fit_time': array([0.77233086, 0.67152948, 0.73942969, 0.74273899, 0.78678741,
        0.72425344, 0.80673678, 0.72626941, 0.82866087, 0.74388728,
        0.8334384 , 0.74620621, 0.81735277, 0.714224  , 0.81575456,
        0.71197946, 0.84078777, 0.7170037 , 0.09123001, 0.11032681,
        0.11012316, 0.10789406, 0.10457311, 0.11086674, 0.10560794,
        0.10639765, 0.10909858, 0.10499191, 0.77965147, 0.10989406]),
 'std_fit_time': array([0.05297136, 0.04824414, 0.08927078, 0.06940731, 0.05755375,
        0.09943079, 0.08814884, 0.04516295, 0.07418463, 0.1000181 ,
        0.0672726 , 0.06316461, 0.08141024, 0.06466874, 0.05241215,
        0.05030272, 0.05917558, 0.0445536 , 0.00582009, 0.01993988,
        0.00615108, 0.00525358, 0.00578845, 0.01488547, 0.00524369,
        0.00510661, 0.01382222, 0.00481068, 0.17654032, 0.02690411]),
 'mean_score_time': array([0.01405358, 0.01212013, 0.01106737, 0.01448443, 0.01525598,
        0.011589  , 0.01308002, 0.01449947, 0.01204782, 0.01

In [113]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 25, 24, 21,  1, 22,  3,  2,  3,  3,  3,  3,  3,  3,  3,
        3, 28, 25,  3, 22,  3,  3,  3,  3,  3,  3,  3,  3])

In [114]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [115]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.600110
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.737000
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.742856
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.745318
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.745827
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.745884
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746011
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.745884
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.745926
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.745940


In [116]:
acc_list.append(results[0:1])

In [117]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'lbfgs'}

In [118]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.709747
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.698080
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.696729
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.694938
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.694541
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.694381
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.694159
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.694194
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.694205


In [119]:
roc_auc_ovr_List.append(results[18:19])

In [120]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.1,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [121]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.600110
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.737000
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.742856
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.745318
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.745827
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.745884
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746011
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.745884
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.745926
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.745940


In [122]:
f1_micro_List.append(results[6:7])

## Trial 10

In [123]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state = 10)

In [124]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWar

In [125]:
best_model.cv_results_

{'mean_fit_time': array([0.78448954, 0.68094416, 0.7873116 , 0.69274485, 0.77690225,
        0.6648411 , 0.82125957, 0.72304151, 0.82754579, 0.77510939,
        0.95549963, 0.80591969, 0.960993  , 0.81434066, 0.91933064,
        0.84659867, 0.96487954, 0.84253166, 0.09054177, 0.09921424,
        0.10849853, 0.10646503, 0.10180056, 0.10059524, 0.11263793,
        0.10580204, 0.10574818, 0.11453967, 0.71409128, 0.10780144]),
 'std_fit_time': array([0.04209798, 0.04405663, 0.07528806, 0.03540744, 0.06747883,
        0.05824731, 0.04803106, 0.05211329, 0.06806008, 0.11173712,
        0.16091462, 0.12192324, 0.1463901 , 0.15664696, 0.17155203,
        0.14320799, 0.16930209, 0.17885433, 0.00481448, 0.00445428,
        0.00995482, 0.00433255, 0.00592979, 0.0044002 , 0.01913359,
        0.00499686, 0.00605197, 0.01189295, 0.03741974, 0.00374943]),
 'mean_score_time': array([0.0130353 , 0.01155958, 0.01142981, 0.01101227, 0.01136267,
        0.01259608, 0.01429851, 0.01235039, 0.01217241, 0.01

In [126]:
best_model.cv_results_['rank_test_accuracy']

array([30, 28, 27, 25,  2,  3,  1, 23,  5, 14, 14,  6,  6,  6,  6, 14,  6,
       13, 28, 25,  3, 23,  6, 14, 14, 14, 14, 14, 14,  6])

In [127]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [128]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_acc'] = best_model.cv_results_['mean_test_accuracy']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.600422
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.737849
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.742899
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.745813
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.746039
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.746039
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746053
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.745968
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.746011
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.745983


In [129]:
acc_list.append(results[0:1])

In [130]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [131]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_precision'] = best_model.cv_results_['mean_test_precision']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.711210
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.698539
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.697574
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.695150
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.694605
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.694423
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.694223
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.694243
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.694219


In [132]:
roc_auc_ovr_List.append(results[1:2])

In [133]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.1,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [134]:
results = pd.DataFrame( best_model.cv_results_['params'] )

results['score_f1_micro'] = best_model.cv_results_['mean_test_f1_micro']
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.600422
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.737849
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.742899
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.745813
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.746039
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.746039
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.746053
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.745968
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.746011
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.745983


In [135]:
f1_micro_List.append(results[6:7])

In [136]:
end = datetime.now()
time_taken = end - start
print('Time: ',time_taken) 

Time:  0:36:28.388900


## Results

In [137]:
accuracy = [x['score_acc'][0] for x in acc_list]
accuracy_c = [x['C'][0] for x in acc_list]
accuracy_penalty = [x['penalty'][0] for x in acc_list]
roc = [x['score_precision'].reset_index(drop=True)[0] for x in roc_auc_ovr_List]
roc_c = [x['C'].reset_index(drop=True)[0] for x in roc_auc_ovr_List]
roc_penalty = [x['penalty'].reset_index(drop=True)[0] for x in roc_auc_ovr_List]
f1 = [x['score_f1_micro'].reset_index(drop=True)[0] for x in f1_micro_List]
f1_c = [x['C'].reset_index(drop=True)[0] for x in f1_micro_List]
f1_penalty = [x['penalty'].reset_index(drop=True)[0] for x in f1_micro_List]

In [138]:
data = {'Accuracy': accuracy, 'Accuracy C': accuracy_c, 'Accuracy Penalty': accuracy_penalty,
        'Roc_auc_ovr': roc, 'Roc_auc_ovr C': roc_c, 'Roc_auc_ovr Penalty': roc_penalty,
        'F1_micro':f1, 'F1_micro C': f1_c, 'F1_micro Penalty': f1_penalty}

In [139]:
trainingResults  = pd.DataFrame(data = data)
pd.options.display.max_colwidth = 100
trainingResults

,Accuracy,Accuracy C,Accuracy Penalty,Roc_auc_ovr,Roc_auc_ovr C,Roc_auc_ovr Penalty,F1_micro,F1_micro C,F1_micro Penalty
0,0.600535,0.0001,l1,0.712024,0.0001,l2,0.746704,0.01,l2
1,0.599120,0.0001,l1,0.712332,0.0001,l2,0.747043,1000.00,l2
2,0.599601,0.0001,l1,0.711806,0.0001,l2,0.746845,0.10,l1
3,0.599332,0.0001,l1,0.712440,0.0001,l2,0.747652,1.00,l2
4,0.600167,0.0001,l1,0.711268,0.0001,l2,0.746987,0.10,l1
5,0.599389,0.0001,l1,0.711137,0.0001,l2,0.746407,10000.00,l1
6,0.599785,0.0001,l1,0.711292,0.0001,l2,0.746464,0.10,l1
7,0.599969,0.0001,l1,0.710429,0.0001,l2,0.746138,1.00,l1
8,0.600110,0.0001,l1,0.709766,0.0001,l2,0.746011,0.10,l1
9,0.600422,0.0001,l1,0.711210,0.0001,l2,0.746053,0.10,l1


In [141]:
trainingResults.to_csv('LR_trainingResults.csv')